In [ ]:
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as albu
import pandas as pd
import numpy as np
import os
import random

import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold

from utils.models import EffNetMSD
from utils.loss import FocalCosineLoss, focal_loss
from utils.trainer import NNTrainer
from utils.data import MyDataset, get_train_transforms, get_valid_transforms

import efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
from torchvision import models

In [ ]:
# """
# "0": "Cassava Bacterial Blight (CBB)", 
# "1": "Cassava Brown Streak Disease (CBSD)", 
# "2": "Cassava Green Mottle (CGM)", 
# "3": "Cassava Mosaic Disease (CMD)", 
# "4": "Healthy"
# """

In [ ]:
cfg = {'model_name' : 'efficientnet-b5', #'resnext50_32x4d',#'resnext101_32x8d', #'resnext50_32x4d', #'efficientnet-b3',
       'advprop' : False,
       'seed' : 256,
       'img_size' : 512,
       'batch_size' : 4,
       'accum_steps' : 1,
       'n_epoch' : 15,
       'patience' : 3,
       'n_dropouts' : 30,
       'tta' : None,
       'lr' : 2e-4,
       'weight_decay' : 0, # 1e-6,
       'extra_data' : True,
       'extra_data_unsepervised' : False,
       'sample_coeffs' : {0:4, 1:2.5, 2:2.5, 3:1, 4:2.5},
       'fullsampling' : None, #True,
       
       'optim' : Adam,

       #loss, weighted = lambda x, y: focal_loss(x, y), False
       #loss, weighted = torch.nn.CrossEntropyLoss, True
       'loss' : FocalCosineLoss(),
#       'loss' : lambda x, y: focal_loss(x, y),
       
       'weighted' : False,
       
       'scheduler' : lambda x : CosineAnnealingWarmRestarts(x, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1),
        
       'random_state' : 123,

       'device' : 'cuda',
}

cfg['augmentations_train'] = get_train_transforms
cfg['augmentations_test'] = cfg['augmentations_train'] #get_valid_transforms(cfg['img_size']), 

In [ ]:
cfg

In [ ]:
path = './'
path2 = './add_data'
path_train = f'{path}/train_images/'
path_dct = {0 : f'{path2}/train/cbb/',
            1 : f'{path2}/train/cbsd/',
            2 : f'{path2}/train/cgm/',
            3 : f'{path2}/train/cmd/',
            4 : f'{path2}/train/healthy/'}
path_extra = f'{path2}/extraimages/'

In [ ]:
random.seed(cfg['seed'])

In [ ]:
df = pd.read_csv('train.csv')
df = df[(df['image_id'] != '3551135685.jpg')&(df['image_id'] != '2252529694.jpg')]

In [ ]:
df2 = pd.read_csv('pl_df.csv')
df2 = df2[['image_id', 'label']]

In [ ]:
df.label.hist()

In [ ]:
df2.label.hist()

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=cfg['random_state'])
splits = skf.split(df.image_id, df.label)

In [ ]:
metrics = []

In [ ]:
for i, (train_idx, valid_idx) in enumerate(splits):
    model = EffNetMSD(net=cfg["model_name"], advprop=cfg['advprop'], n_dropouts=cfg['n_dropouts'])
    old_model = './models_pl/eff5_2/efficientnet-b5_split_0_best_.pth'
    model.load_state_dict(torch.load(old_model, map_location=cfg['device']))
    
    if False:  #'efficientnet' in cfg["model_name"]:
        for name, param in model.named_parameters():
            if '_bn' in name:
                param.requires_grad = False
            else:
                param.requires_grad = True
    else:        
        for param in model.parameters():
            param.requires_grad = True

    if not cfg['extra_data']:
        path_dct=None
        
    train_dataset = MyDataset(train_df=df, train_idx=train_idx, train_path=path_train,\
                              train_df2=df2, train_path2=path_extra, img_size=cfg['img_size'],\
                              augmentations=cfg['augmentations_train'](cfg['img_size']),\
                              path_dct=path_dct, shuffle=cfg['random_state'], advprop=cfg['advprop'],\
                              sample_coeffs=cfg['sample_coeffs'], fullsampling=cfg['fullsampling'])
    
    valid_dataset = MyDataset(train_df=df, train_idx=valid_idx, train_path=path_train, img_size=cfg['img_size'],\
                              augmentations=cfg['augmentations_test'](cfg['img_size']),\
                              shuffle=cfg['random_state'], advprop=cfg['advprop'], tta=cfg['tta'])
    extra_dataset = MyDataset(path_extra=path_extra, path_extra2=None, img_size=cfg['img_size'],\
                              augmentations=cfg['augmentations_train'](cfg['img_size']),\
                              augmentations2=cfg['augmentations_test'](cfg['img_size']),\
                              shuffle=cfg['random_state'], advprop=cfg['advprop'])
    
    train_loader = DataLoader(train_dataset, batch_size=cfg['batch_size'], drop_last=False, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=cfg['batch_size'], drop_last=False, shuffle=False)
    if cfg['extra_data_unsepervised']:
        extra_loader = DataLoader(extra_dataset, batch_size=cfg['batch_size'], drop_last=False, shuffle=True)
    else:
        extra_loader = None
    
    trainer = NNTrainer(model, device=cfg['device'], label_names=[0, 1, 2, 3, 4], model_name=f'{cfg["model_name"]}_split_{i}')
    metric = trainer.train(train_loader=train_loader, val_loader=valid_loader, extra_loader=extra_loader,\
                  n_epoch=cfg['n_epoch'], optim=cfg['optim'], weight_decay=cfg['weight_decay'], schedul=cfg['scheduler'],\
                  loss=cfg['loss'], weighted=cfg['weighted'], lr=cfg['lr'], accum_steps=cfg['accum_steps'], show_results=True,\
                  saved_models_dir='./models/', verbose=True, patience=cfg['patience'], calc_metric_batch=None)
    
    metrics.append(metric)
    print('best metrics:', metrics)
    print('mean best metrics:', np.mean(metrics))
    
    model.to('cpu')

    del trainer
    del model
    torch.cuda.empty_cache()